In [ ]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()

if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab-ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-lg9ry05a/unsloth_b3ad7a47600f4688946bdc54cce50439
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-lg9ry05a/unsloth_b3ad7a47600f4688946bdc54cce50439
  Resolved https://github.com/unslothai/unsloth.git to commit 52bc19d1fa4cd3557b785127fd68b5f4d1c34347
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://download.pytorch.org/whl/cu121/xformers-0.0.24-cp310-cp310-manylinux2014_x86_64.whl (218.2 MB)


In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048

dtype = None

load_in_4bit = True

# 모델 설정
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/content/drive/MyDrive/mbti-senti_model",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    cache_dir='/data'
)

Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.24. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('/content/drive/MyDrive/csvs/news_summ.csv', encoding='UTF-8')

dataset = Dataset.from_pandas(df)

dataset

Dataset({
    features: ['news', 'summary'],
    num_rows: 42803
})

In [ ]:
def formatting_summ_prompts_func(examples):
    output_texts = []
    for i in range(len(examples['news'])):
        messages = [
            {"role": "system", "content": "정확한 챗봇으로서 상대방의 입력에 대해 요약을 하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
            {"role": "user", "content": "{}".format(examples['news'][i])},
            {"role": "assistant", "content": "{}".format(examples['summary'][i])}
        ]
        # LLAMA의 모델 클래스에 맞는 기본 채팅 템플릿 적용
        chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        output_texts.append(chat_message)

    return {"text": output_texts}

In [ ]:
formatted_dataset = dataset.map(formatting_summ_prompts_func, batched=True)

Map:   0%|          | 0/42803 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,  # 멀티프로세싱을 위해 더 많은 프로세스를 사용 (코랩에서 사용할 수 있는 리소스에 따라 조정)
    packing=False,  # 짧은 시퀀스의 경우 False가 더 나을 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # GPU 메모리에 맞춰 배치 크기 조정
        gradient_accumulation_steps=4,  # 배치 크기를 효과적으로 늘리기 위해 그래디언트 누적 사용
        warmup_steps=100,  # 워밍업 스텝을 늘려 모델이 더 안정적으로 학습하도록 설정
        max_steps=1000,  # 학습 스텝 수를 늘려 충분한 학습 시간 확보
        learning_rate=5e-5,  # 보편적으로 사용되는 학습률
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps=50,  # 로그를 덜 빈번하게 출력하여 학습 속도에 영향을 주지 않도록 함
        optim="adamw_torch",  # 최신 옵티마이저 사용 (PyTorch의 기본 AdamW 구현)
        weight_decay=0.01,  # 일반적으로 사용되는 weight decay
        lr_scheduler_type="cosine",  # 학습률이 부드럽게 감소하도록 cosine 스케줄 사용
        seed=3407,
        output_dir="summ_outputs",
        save_steps=100,  # 체크포인트를 자주 저장하여 중간에 학습을 중단하고도 재개 가능하도록 설정
    ),
)

trainer_stats = trainer.train()

Map (num_proc=2):   0%|          | 0/42803 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 42,803 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

In [ ]:
import os
import shutil

source_path      = '/content/model'
destination_path = '/content/drive/MyDrive/mbti-senti-summ_model'

if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# source_path 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.move(file_path, destination_path)

In [ ]:
source_path2      = '/content/outputs'
destination_path2 = '/content/drive/MyDrive/summ_outputs'

if not os.path.exists(destination_path2):
    os.makedirs(destination_path2)

# source_path 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path2):
    file_path = os.path.join(source_path2, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.move(file_path, destination_path2)